In [1]:
def setHadoopConfigd3bd5b94a9334de59a55a7fed2bedeaa(name: String) = {
    // This function sets the Hadoop configuration so it is possible to
    // access data from Bluemix Object Storage using Spark

    val prefix = "fs.swift.service." + name
    sc.hadoopConfiguration.set(prefix + ".auth.url", "https://identity.open.softlayer.com" + "/v3/auth/tokens")
    sc.hadoopConfiguration.set(prefix + ".auth.endpoint.prefix","endpoints")
    sc.hadoopConfiguration.set(prefix + ".tenant", "6aaf54352357483486ee2d4981f8ef15")
    sc.hadoopConfiguration.set(prefix + ".username", "9fea4ce0afb1415aac1a9c2dffb84be4")
    sc.hadoopConfiguration.set(prefix + ".password", "o[R4n/*RSL7x3Bjj")
    sc.hadoopConfiguration.setInt(prefix + ".http.port", 8080)
    sc.hadoopConfiguration.set(prefix + ".region", "dallas")
    sc.hadoopConfiguration.setBoolean(prefix + ".public", false)
}
val name = "keystone"
setHadoopConfigd3bd5b94a9334de59a55a7fed2bedeaa(name)
var df_categorical = spark.read.parquet("swift://sparkbuch." + name + "/train_categorical2.0.parquet")
var df_numeric = spark.read.parquet("swift://sparkbuch." + name + "/train_numeric2.0.parquet")

In [3]:
df_categorical.printSchema
df_numeric.printSchema

root
 |-- Id: integer (nullable = true)
 |-- L0_S1_F25: string (nullable = true)
 |-- L0_S1_F27: string (nullable = true)
 |-- L0_S1_F29: string (nullable = true)
 |-- L0_S1_F31: string (nullable = true)
 |-- L0_S2_F33: string (nullable = true)
 |-- L0_S2_F35: string (nullable = true)
 |-- L0_S2_F37: string (nullable = true)
 |-- L0_S2_F39: string (nullable = true)
 |-- L0_S2_F41: string (nullable = true)
 |-- L0_S2_F43: string (nullable = true)
 |-- L0_S2_F45: string (nullable = true)
 |-- L0_S2_F47: string (nullable = true)
 |-- L0_S2_F49: string (nullable = true)
 |-- L0_S2_F51: string (nullable = true)
 |-- L0_S2_F53: string (nullable = true)
 |-- L0_S2_F55: string (nullable = true)
 |-- L0_S2_F57: string (nullable = true)
 |-- L0_S2_F59: string (nullable = true)
 |-- L0_S2_F61: string (nullable = true)
 |-- L0_S2_F63: string (nullable = true)
 |-- L0_S2_F65: string (nullable = true)
 |-- L0_S2_F67: string (nullable = true)
 |-- L0_S3_F69: string (nullable = true)
 |-- L0_S3_F71: s

In [4]:
df_numeric.first

[4,0.03,-0.034,-0.197,-0.179,0.118,0.116,-0.015,-0.032,0.02,0.083,-0.273,-0.273,-0.271,0.167,-0.213,-0.023,-0.192,-0.088,0.001,0,0.01,-0.223,-0.03,null,null,null,null,null,null,null,null,null,-0.001,-0.004,null,null,null,null,null,-0.164,-0.077,0.06,-0.157,0.0,0.001,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.018,-0.136,-0.013,0.106,-0.112,-0.131,-0.021,-0.097,0.26,0.012,-0.115,0.113,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,nul...

In [5]:
df_categorical.createOrReplaceTempView("dfcat")
var dfcat = spark.sql("select Id, L0_S22_F545 from dfcat")

In [6]:
df_numeric.createOrReplaceTempView("dfnum")
var dfnum = spark.sql("select Id,L0_S0_F0,L0_S0_F2,L0_S0_F4,Response from dfnum")


In [18]:
var df = dfcat.join(dfnum,"Id")
df.createOrReplaceTempView("df")

var df_notnull = spark.sql("""
select
    Response as label,
    case 
       when L0_S22_F545 is null then 'NA'
       when L0_S22_F545 = '' then 'NA' 
       else L0_S22_F545 end as L0_S22_F545, 
    case
       when L0_S0_F0 is null then 0.0 
       else L0_S0_F0 end as L0_S0_F0, 
    case
       when L0_S0_F2 is null then 0.0 
       else L0_S0_F2 end as L0_S0_F2,
    case
       when L0_S0_F4 is null then 0.0 
       else L0_S0_F4 end as L0_S0_F4
from df
""")

In [19]:
import org.apache.spark.ml.feature.{OneHotEncoder, StringIndexer}

val indexer = new StringIndexer(). 
  setInputCol("L0_S22_F545"). 
  setOutputCol("L0_S22_F545Index")

val indexed = indexer.setHandleInvalid("skip").fit(df_notnull).transform(df_notnull)


In [20]:
indexed.printSchema

root
 |-- label: integer (nullable = true)
 |-- L0_S22_F545: string (nullable = true)
 |-- L0_S0_F0: double (nullable = true)
 |-- L0_S0_F2: double (nullable = true)
 |-- L0_S0_F4: double (nullable = true)
 |-- L0_S22_F545Index: double (nullable = true)



In [21]:
indexed.first

[0,NA,0.03,-0.034,-0.197,0.0]

In [22]:
indexed.select("L0_S22_F545Index").show

+----------------+
|L0_S22_F545Index|
+----------------+
|             0.0|
|             0.0|
|             0.0|
|             0.0|
|             0.0|
|             0.0|
|             0.0|
|             0.0|
|             0.0|
|             0.0|
|             0.0|
|             0.0|
|             0.0|
|             0.0|
|             0.0|
|             0.0|
|             0.0|
|             0.0|
|             0.0|
|             0.0|
+----------------+
only showing top 20 rows



In [27]:
var encoder = new OneHotEncoder().setInputCol("L0_S22_F545Index").setOutputCol("L0_S22_F545Vec")
val encoded = encoder.transform(indexed)

In [29]:
encoded.printSchema

root
 |-- label: integer (nullable = true)
 |-- L0_S22_F545: string (nullable = true)
 |-- L0_S0_F0: double (nullable = true)
 |-- L0_S0_F2: double (nullable = true)
 |-- L0_S0_F4: double (nullable = true)
 |-- L0_S22_F545Index: double (nullable = true)
 |-- L0_S22_F545Vec: vector (nullable = true)



In [30]:
encoded.first

[0,NA,0.03,-0.034,-0.197,0.0,(13,[0],[1.0])]

In [31]:
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vectors

val transformers = Array(
    indexer,
    encoder,
    new VectorAssembler()
        .setInputCols(Array("L0_S22_F545Vec", "L0_S0_F0", "L0_S0_F2","L0_S0_F4"))
        .setOutputCol("features")
)

In [32]:
var pipeline = new Pipeline().setStages(transformers).fit(df_notnull)

var transformed = pipeline.transform(df_notnull)

In [33]:
transformed.printSchema

root
 |-- label: integer (nullable = true)
 |-- L0_S22_F545: string (nullable = true)
 |-- L0_S0_F0: double (nullable = true)
 |-- L0_S0_F2: double (nullable = true)
 |-- L0_S0_F4: double (nullable = true)
 |-- L0_S22_F545Index: double (nullable = true)
 |-- L0_S22_F545Vec: vector (nullable = true)
 |-- features: vector (nullable = true)



In [34]:
transformed.first

[0,NA,0.03,-0.034,-0.197,0.0,(13,[0],[1.0]),(16,[0,13,14,15],[1.0,0.03,-0.034,-0.197])]

In [35]:
import org.apache.spark.ml.classification.RandomForestClassifier
val rf = new RandomForestClassifier(). 
  setLabelCol("label").
  setFeaturesCol("features")

val model = new Pipeline().setStages(transformers :+ rf).fit(df_notnull)

val result = model.transform(df_notnull)

In [36]:
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
val evaluator = new BinaryClassificationEvaluator()

import org.apache.spark.ml.param.ParamMap
var evaluatorParamMap = ParamMap(evaluator.metricName -> "areaUnderROC")
var aucTraining = evaluator.evaluate(result, evaluatorParamMap)

In [39]:
print(aucTraining)

0.5442689365296324

In [41]:
import org.apache.spark.ml.tuning.{CrossValidator, ParamGridBuilder}
/**var paramGrid = new ParamGridBuilder()
    .addGrid(rf.numTrees, 3 :: 5 :: 10 :: 30 :: 50 :: 70 :: 100 :: 150 :: Nil)
    .addGrid(rf.featureSubsetStrategy, "auto" :: "all" :: "sqrt" :: "log2" :: "onethird" :: Nil)
    .addGrid(rf.impurity, "gini" :: "entropy" :: Nil)    
    .addGrid(rf.maxBins, 2 :: 5 :: 10 :: 15 :: 20 :: 25 :: 30 :: Nil)
    .addGrid(rf.maxDepth, 3 :: 5 :: 10 :: 15 :: 20 :: 25 :: 30 :: Nil)
    .build()*/

var paramGrid = new ParamGridBuilder().
    addGrid(rf.numTrees, 3 :: 5 :: 10 :: Nil).
    addGrid(rf.featureSubsetStrategy, "auto" :: "all" :: Nil).
    addGrid(rf.impurity, "gini" :: "entropy" :: Nil).    
    addGrid(rf.maxBins, 2 :: 5 :: Nil).
    addGrid(rf.maxDepth, 3 :: 5 :: Nil).
    build()

In [43]:
var crossValidator = new CrossValidator().
      setEstimator(new Pipeline().setStages(transformers :+ rf)).
      setEstimatorParamMaps(paramGrid).
      setNumFolds(5).
      setEvaluator(evaluator)

In [ ]:
//Model is created
    var crossValidatorModel = crossValidator.fit(df_notnull)
    //Model used to Predict
    var newPredictions = crossValidatorModel.transform(df_notnull)


    var newAucTest = evaluator.evaluate(newPredictions, evaluatorParamMap)
    println("new AUC (with Cross Validation) " + newAucTest)
    var bestModel = crossValidatorModel.bestModel

    //Understand the Model selected
    println()
    println("Parameters for Best Model:")

    var bestPipelineModel = crossValidatorModel.bestModel.asInstanceOf[PipelineModel]
    var stages = bestPipelineModel.stages

In [ ]:
import org.apache.spark.ml.classification.RandomForestClassificationModel
    val rfStage = stages(stages.length-1).asInstanceOf[RandomForestClassificationModel]
rfStage.getNumTrees
rfStage.getFeatureSubsetStrategy
rfStage.getImpurity
rfStage.getMaxBins
rfStage.getMaxDepth